In [5]:
import gurobipy as gp
import numpy as np
import pandas as pd
import numpy.linalg as la
from gurobipy import GRB
import plotly.express as px
import datetime

import quarters
import companies
import dates

In [6]:
date = datetime.datetime(2023, 10, 15)
dates.nearest_trading_date(date, 'nearest')

Timestamp('2023-10-16 00:00:00-0400', tz='America/New_York')

In [7]:
def get_stock_weights(start_year, end_year, a, min_return):
    model = gp.Model('portfolio')
    model.params.NonConvex = 2
    model.setParam('NonConvex', 2)

    rfr = quarters.get_mean_rfr_between_years(start_year, end_year)
    gdp = quarters.get_gdp_growth_between_years(start_year, end_year)
    
    returns = pd.read_csv(f'./data/returns/returns_{start_year}-{end_year}.csv').set_index('Unnamed: 0')

    mu1 = returns.mean().to_numpy()
    mu2 = np.array([np.mean(rfr), np.mean(gdp)])
    #print(mu2)
    all_companies = returns.columns
    n_companies = returns.shape[1]

    #adding external parameters
    returns['avg_rfr'] = rfr
    returns['gdp_growth'] = gdp

    total_points = returns.shape[1]
    
    returns_array = returns.to_numpy().T.astype(float)
    cov_array = np.cov(returns_array)
    #print(cov_array)
    sigma11 = cov_array[0:n_companies, 0:n_companies]
    sigma21 = cov_array[n_companies:total_points, 0:n_companies]
    sigma12 = cov_array[0:n_companies, n_companies:total_points]
    sigma22 = cov_array[n_companies:total_points, n_companies:total_points]
    #print(sigma12, sigma22)
    mu = mu1 + sigma12@la.inv(sigma22)@(a-mu2)
    sigma = sigma11 - sigma12@la.inv(sigma22)@sigma21

    w = model.addMVar(n_companies, name='weights', lb=0)

    model.addConstr(mu@w == min_return, name='min_returns')
    model.addConstr(w.sum() ==  1, name='integrity')
    
    
    model.setObjective(w@sigma@w, GRB.MINIMIZE)
    model.write('./new_sol.lp')
    model.optimize()

    try:
        weights = w.X
        obj = model.ObjVal
    except:
        weights = None
        obj = 0
    return (weights, obj)

In [8]:
from dev_email import send_email
import timeit

start = timeit.default_timer()

exp_array = [(i+50)/1000 for i in range(0, 100)]

results_dict = {}

for expectation in exp_array:
    results = get_stock_weights(2010, 2019, np.array([0.023734, 1.329796395805503/100]), expectation)
    variance = results[1]
    weights = results[0]
    results_dict[expectation] = {'variance':variance, 'weights':weights}

time = end-start
#send_email(f'\"Solving for optimal weights between 1995 and 2019. Took {time/60} minutes\"')

NameError: name 'end' is not defined

In [4]:
results_dict

{0.05: {'variance': 0.0007504889651943794,
  'weights': array([3.80854217e-09, 1.26264386e-08, 4.28815870e-09, 8.21597252e-09,
         1.20001235e-08, 4.61479367e-09, 5.73603931e-09, 4.09143834e-09,
         6.33085427e-09, 5.59596266e-09, 3.08028913e-09, 7.60589157e-09,
         7.13040388e-09, 4.92690800e-02, 1.28431604e-08, 1.35174264e-08,
         1.27310108e-08, 6.10403606e-09, 1.82719281e-01, 2.41896133e-09,
         1.14494018e-08, 3.89848066e-09, 5.30042854e-09, 9.71571765e-09,
         3.57715880e-09, 1.55619559e-08, 3.72194329e-09, 1.15175964e-08,
         4.84772441e-09, 5.71703645e-09, 4.54263175e-07, 1.73792016e-07,
         2.67999138e-09, 1.39023938e-08, 7.42109194e-09, 1.65359565e-08,
         5.62014448e-09, 4.58610267e-09, 2.93266463e-09, 5.73868690e-09,
         7.03803896e-09, 1.75449892e-07, 3.58075132e-09, 1.24506947e-08,
         4.75884398e-09, 6.31688349e-09, 1.38469305e-08, 5.92250364e-09,
         1.57787154e-08, 1.43009902e-08, 9.98327529e-09, 4.39598943e-0

In [11]:
for result in results_dict:
    variance = results_dict[result]['variance']
    if variance == 0:
        continue
    sharpe_ratio = (result)/variance
    print(result, sharpe_ratio)

0.05 20.633541040235183
0.051 19.901041147888098
0.052 19.149391947719657
0.053 18.3925159397118
0.054 17.641608135872445
0.055 16.907044630226046
0.056 16.195455367391993
0.057 15.511479623478742
0.058 14.855146709877596
0.059 14.226276696997637
0.06 13.625975869023712
0.061 13.054293123651311
0.062 12.51019375533549
0.063 11.993336547955966
0.064 11.50311284195962
0.065 11.038755003053543
0.066 10.59900503657047
0.067 10.18188341163757
0.068 9.786055652396263
0.069 9.409771132363755
0.07 9.051722035413112
0.071 8.711363378579037
0.072 8.388070869624986
0.073 8.0811218064244
0.074 7.788829444436825
0.075 7.510158199171367
0.076 7.243972456725576
0.077 6.989689849447113
0.078 6.746978856994828
0.079 6.515363530459123
0.08 6.294426339599217
0.081 6.083765606660085
0.082 5.882760606795922
0.083 5.6887224371133716
0.084 5.499117696338248
0.085 5.306245495854146
0.086 5.111564910302046
0.087 4.917371498197387
0.088 4.7255384027447125
0.089 4.537550504581956
0.09 4.354547112915141
0.091 4.1

In [7]:
benchmark = yf.Ticker('SPY').history(start=start_date_local)['Close']
total_portfolio = pd.DataFrame(index=benchmark.index)
total_portfolio['portfolio'] = markowitz.sum(axis=1)
total_portfolio['benchmark'] = benchmark/benchmark.iloc[0]
total_portfolio

NameError: name 'yf' is not defined

In [8]:
import yfinance as yf
from dev_email import send_email
import datetime

start_year_local = 2019
start_date_local = datetime.datetime(year=start_year_local, month=1, day=1)


returns = pd.read_csv(f'./data/returns/returns_1995-2019.csv').set_index('Unnamed: 0')
all_companies = returns.columns

markowitz = pd.DataFrame(index=yf.Ticker('AAPL').history(start=start_date_local).index)

for i in range(len(all_companies)):
    
    company = all_companies[i]
    weight = results[i]
    history = yf.Ticker(company).history(start=start_date_local)['Close']
    
    adjusted = history/history.iloc[0]*weight
    
    markowitz[company] = adjusted

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [6]:
plot = px.line(total_portfolio)
plot.show()

NameError: name 'total_portfolio' is not defined

In [4]:
results_dict

{0.05: {'variance': 0.002423238934243063,
  'weights': array([3.90590706e-11, 2.33900546e-11, 7.14577236e-12, 2.06716797e-11,
         2.12412469e-11, 3.87687759e-11, 1.12652771e-11, 5.88298556e-11,
         9.56490988e-12, 5.65821961e-02, 1.05929893e-01, 1.18704068e-11,
         1.04342020e-11, 2.83381625e-11, 5.25672032e-02, 8.67153968e-11,
         8.58234653e-11, 1.73653935e-11, 1.45112256e-11, 3.46134284e-10,
         9.31570773e-12, 1.57971835e-11, 2.32124726e-09, 1.31634888e-11,
         1.47792093e-10, 4.03801961e-11, 3.67784893e-02, 1.06031608e-11,
         2.62747538e-11, 5.14489894e-12, 9.59096304e-12, 1.20897359e-11,
         8.43251696e-12, 3.26733321e-09, 1.92065680e-11, 4.53932746e-02,
         1.08071024e-10, 1.67704317e-10, 1.85426325e-11, 5.54794376e-10,
         1.64017400e-11, 2.62816910e-11, 3.74857832e-11, 1.73413975e-11,
         2.13817236e-11, 2.57616436e-11, 1.00380216e-08, 3.40702197e-11,
         5.86875634e-02, 1.20499779e-11, 1.60684750e-10, 3.66311708e-02